In [8]:
import os
HUGGINGFACE_TOKEN = "agsdgsdgesdagasdfasdfasfd" #@param {type:"string"}
os.environ['HF_TOKEN']=HUGGINGFACE_TOKEN

#################################################################

In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import LoraConfig, get_peft_model

# Set random seed for reproducibility
torch.random.manual_seed(0)

# -----------------------------------------------------------------------------
# Load the Phi-3-mini-4k-instruct model and tokenizer.
# Using "cpu" here; if you have a GPU, you might change the device_map to "auto".
# -----------------------------------------------------------------------------
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cpu",           # Change to "auto" if running on a GPU-enabled machine.
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
# Apply LoRA configuration using PEFT to enable parameter-efficient fine-tuning.
# This wraps the original model with LoRA layers.
# -----------------------------------------------------------------------------
for name, module in model.named_modules():
    if "proj" in name or "attn" in name:
        print(name)

# -----------------------------------------------------------------------------
# Update the LoRA configuration: Since your inspection shows layers like:
#   model.layers.<number>.self_attn.qkv_proj
# we'll use "self_attn.qkv_proj" as the target module.
# -----------------------------------------------------------------------------
lora_config = LoraConfig(
    r=8,                         # LoRA rank.
    lora_alpha=32,               # Scaling factor.
    target_modules=["self_attn.qkv_proj"],  # Updated target module.
    lora_dropout=0.05,           # Dropout rate for LoRA layers.
    bias="none",
)
model = get_peft_model(model, lora_config)

model.layers.0.self_attn
model.layers.0.self_attn.o_proj
model.layers.0.self_attn.qkv_proj
model.layers.0.self_attn.rotary_emb
model.layers.0.mlp.gate_up_proj
model.layers.0.mlp.down_proj
model.layers.0.resid_attn_dropout
model.layers.1.self_attn
model.layers.1.self_attn.o_proj
model.layers.1.self_attn.qkv_proj
model.layers.1.self_attn.rotary_emb
model.layers.1.mlp.gate_up_proj
model.layers.1.mlp.down_proj
model.layers.1.resid_attn_dropout
model.layers.2.self_attn
model.layers.2.self_attn.o_proj
model.layers.2.self_attn.qkv_proj
model.layers.2.self_attn.rotary_emb
model.layers.2.mlp.gate_up_proj
model.layers.2.mlp.down_proj
model.layers.2.resid_attn_dropout
model.layers.3.self_attn
model.layers.3.self_attn.o_proj
model.layers.3.self_attn.qkv_proj
model.layers.3.self_attn.rotary_emb
model.layers.3.mlp.gate_up_proj
model.layers.3.mlp.down_proj
model.layers.3.resid_attn_dropout
model.layers.4.self_attn
model.layers.4.self_attn.o_proj
model.layers.4.self_attn.qkv_proj
model.layers.4.self_a

In [14]:
# -----------------------------------------------------------------------------
# Create a single text-generation pipeline instance to avoid reinitialization delays.
# -----------------------------------------------------------------------------
gen_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

# -----------------------------------------------------------------------------
# Functions for generating a question and candidate answers from context
# -----------------------------------------------------------------------------
def generate_question(context):
    """
    Generate one clear, insightful question based on the provided context.
    """
    prompt = (
        f"Read the following context and generate one clear, insightful question "
        f"that can be answered using this information:\n\nContext:\n{context}\n\nQuestion:"
    )
    generation_args = {
        "max_new_tokens": 100,
        "return_full_text": False,
        "temperature": 0.0,  # Deterministic generation.
        "do_sample": False,
        "use_cache": False,
    }
    output = gen_pipe(prompt, **generation_args)
    question = output[0]['generated_text'].strip()
    return question

def generate_candidate_answers(context, question, num_candidates=5):
    """
    Generate multiple candidate answers for the given question based on the context.
    """
    prompt = (
        f"Using the following context, answer the question in a clear and accurate manner.\n\n"
        f"Context:\n{context}\n\nQuestion:\n{question}\n\nAnswer:"
    )
    generation_args = {
        "max_new_tokens": 250,
        "do_sample": True,                # Enable sampling for diverse answers.
        "num_return_sequences": num_candidates,
        "temperature": 0.7,               # Moderate temperature for balanced creativity.
        "return_full_text": False,
        "use_cache": False,
    }
    outputs = gen_pipe(prompt, **generation_args)
    candidate_answers = [out['generated_text'].strip() for out in outputs]
    return candidate_answers

Device set to use cpu
The model 'PeftModel' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausa

In [15]:
def select_best_answer(candidate_answers):
    """
    Select the best answer based on a simple heuristic:
    Choose the candidate whose word count is closest to the average word count.
    """
    word_counts = [len(answer.split()) for answer in candidate_answers]
    avg_count = sum(word_counts) / len(word_counts)
    best_index = min(range(len(word_counts)), key=lambda i: abs(word_counts[i] - avg_count))
    return candidate_answers[best_index], best_index

# -----------------------------------------------------------------------------
# Example usage with a wiki-style context (e.g. Roger Federer's early life)
# -----------------------------------------------------------------------------
context = """
Roger Federer was born on August 8, 1981, in Basel, Switzerland. From an early age, he showed great talent
in sports, particularly in tennis. As a child, he played various sports and quickly developed a passion
for tennis, influenced by the local culture and a family that valued athletic achievement. His early
experiences on the court laid the foundation for what would become one of the greatest careers in the history
of tennis.
"""

# Step 1: Generate a question from the context
question = generate_question(context)
print("Generated Question:")
print(question)
print("\n" + "-" * 50 + "\n")

# Step 2: Generate 5 candidate answers for the generated question
candidate_answers = generate_candidate_answers(context, question, num_candidates=5)
print("Candidate Answers:")
for idx, ans in enumerate(candidate_answers, start=1):
    print(f"Candidate {idx}:")
    print(ans)
    print()

# Step 3: Select the best answer based on our heuristic
best_answer, best_index = select_best_answer(candidate_answers)
print("Selected Best Answer (Candidate {}):".format(best_index + 1))
print(best_answer)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


KeyboardInterrupt: 